<h1> Earthquake Cycle Simulations

In [ ]:
# Plot friction parameters
using Gadfly

fricPlot = plot(layer(x = (cca-ccb).*1e3 + 10, y = FltX/1e3,Theme(default_color=colorant"red"), Geom.point),
                layer(x = Seff/1e6, y = FltX/1e3, Theme(default_color=colorant"deepskyblue"), Geom.point),
                layer(x = tauo/1e6, y = FltX/1e3, Theme(default_color=colorant"green"), Geom.point),
                Guide.xlabel("Scaled (a-b)/ Stress value"),
                Guide.ylabel("Depth (m)"),
                Guide.title("Rate and state friction/Stress"),
                Coord.Cartesian(ymin=-24))

In [34]:
@time include("src/setup.jl")
@time include("src/timesolver.jl")

Total number of nodes on fault = 121
Average node spacing = 400.0
dt = 0.04486312014474891


Setup Complete
  2.773581 seconds (2.98 M allocations: 465.026 MiB, 4.50% gc time)


LoadError: LoadError: [91mDimensionMismatch("cannot broadcast array to have fewer dimensions")[39m
while loading /Volumes/GoogleDrive/My Drive/Work/Code/JuliaSEMCycle/src/timesolver.jl, in expression starting on line 19

In [40]:
iFlt

121×1 Array{Int32,2}:
   1
   2
   3
   4
   5
  26
  27
  28
  29
  46
  47
  48
  49
   ⋮
 546
 547
 548
 549
 566
 567
 568
 569
 586
 587
 588
 589

In [33]:
Vf0

121×1 Array{Float64,2}:
 1.10984e-9
 1.10984e-9
 1.10984e-9
 1.10984e-9
 1.10984e-9
 1.10984e-9
 1.10984e-9
 1.10984e-9
 1.10984e-9
 1.10984e-9
 1.10984e-9
 1.10984e-9
 1.10984e-9
 ⋮         
 0.001     
 0.001     
 0.001     
 0.001     
 0.001     
 0.001     
 0.001     
 0.001     
 0.001     
 0.001     
 0.001     
 0.001     

In [29]:
2*v[iFlt] + Vpl


121×1 Array{Float64,2}:
 1.10984e-9
 1.10984e-9
 1.10984e-9
 1.10984e-9
 1.10984e-9
 1.10984e-9
 1.10984e-9
 1.10984e-9
 1.10984e-9
 1.10984e-9
 1.10984e-9
 1.10984e-9
 1.10984e-9
 ⋮         
 0.001     
 0.001     
 0.001     
 0.001     
 0.001     
 0.001     
 0.001     
 0.001     
 0.001     
 0.001     
 0.001     
 0.001     

In [ ]:
IDstate

In [ ]:
a

In [ ]:
include("src/PCG.jl")
include("src/dtevol.jl")
include("src/NRsearch.jl")
include("src/otherFunctions.jl")

vPre = v[:]
        dPre = d[:]

        Vf0 = 2*v[iFlt] + Vpl
        Vf = Vf0[:];

In [ ]:
temp

<h3> PCG debug